### Collect the data(tech news)  by web scraping.

In [162]:
import urllib2

In [163]:
from bs4 import BeautifulSoup

In [164]:
def get_all_doxy_donkey_posts(url, links):
    request = urllib2.Request(url)
    response = urllib2.urlopen(request)
    soup = BeautifulSoup(response)
    for a in soup.findAll('a'):
        try:
            url = a['href']
            title = a['title']
            if title == "Older Posts":
                print title, url
                links.append(url)
                get_all_doxy_donkey_posts(url, links)
        except:
            title = ""
    return

blog_url = "http://doxydonkey.blogspot.in/"
links = []
get_all_doxy_donkey_posts(blog_url, links)

Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-23T19:53:00-07:00&max-results=7
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-14T19:02:00-07:00&max-results=7&start=7&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-05-02T19:43:00-07:00&max-results=7&start=14&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-04-17T19:26:00-07:00&max-results=7&start=21&by-date=false
Older Posts http://doxydonkey.blogspot.in/search?updated-max=2017-04-10T18:56:00-07:00&max-results=7&start=28&by-date=false


In [165]:
# get all the text from the post
def get_doxy_donkey_text(test_url):
    request = urllib2.Request(test_url)
    response = urllib2.urlopen(request)
    soup = BeautifulSoup(response)
    mydivs = soup.findAll("div", {"class": "post-body"})
    
    posts = []
    for div in mydivs:
        posts += map(lambda p:p.text.encode('ascii', errors='replace').replace("?", " "), div.findAll("li"))
    return posts

In [177]:
blog_posts = []
for link in links:
    blog_posts += get_doxy_donkey_text(link)

In [178]:
# display all the blog post
blog_posts

["SoftBank's $100 Billion Tech Fund Rankles VCs as Valuations Soar: In the months since Softbank Group Corp. unveiled plans for a $100 billion technology fund, the Japanese company has been making its presence deeply felt across the industry. The Vision Fund closed a few days ago with $93 billion in initial commitments, and already venture firms from London to Silicon Valley are fretting about a behemoth with the resources, clout and name recognition to snatch away the most promising deals. Just last week, SoftBank swooped in and pumped $1.4 billion into Paytm, India s largest digital-payments startup. The deal boosted Paytm's valuation by about 40 percent to $7 billion. That's not outlandish given Paytm's dominant market position, but the valuations of other SoftBank deals have prompted head-scratching and ignited alarm that a funding atmosphere that only recently cooled off will heat up again. there's the concern that SoftBank will ladle out more money than startups need or can absor

In [179]:
# Number of posts we scrapped
len(blog_posts)

102

# Clustering

In [180]:
from sklearn.feature_extraction.text import TfidfVectorizer  # convert text to TF-IDF representation

In [181]:
# Converting into numeric attributes
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')

In [182]:
X = vectorizer.fit_transform(blog_posts)
X  # below output shows 60 articles and 1151 is the no. of distinct words

<102x1706 sparse matrix of type '<type 'numpy.float64'>'
	with 7874 stored elements in Compressed Sparse Row format>

### Making clusters

In [192]:
from sklearn.cluster import KMeans
# belopunctuationw init param means method of choosing initia centroids, 
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, verbose=True) 

In [193]:
km.fit(X)

Initialization complete
Iteration  0, inertia 174.830
Iteration  1, inertia 92.858
Iteration  2, inertia 92.791
Converged at iteration 2: center shift 0.000000e+00 within tolerance 5.533563e-08


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [194]:
import numpy as np
np.unique(km.labels_, return_counts=True)  # labels_ array of cluster no. assigned to each article

(array([0, 1, 2], dtype=int32), array([24, 45, 33]))

### Aggerage text into cluster

In [195]:
text = {}
# labels_ is a tuple, where 1st element is an index of the article in the list of posts and the 2nd element is
# the cluster label, i.e. 0,1,2
for i, cluster in enumerate(km.labels_):
    one_document = blog_posts[i]
    if cluster not in text.keys():
        text[cluster] = one_document
    else:
        text[cluster] += one_document

In [147]:
#print(text)

### Find out the most important words

In [196]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk

In [197]:
_stopwords = set(stopwords.words('english') + list(punctuation) + ["million", "billion", "year", "millions", "billions", "y/y", "'s", "''"])

In [200]:
# top keywords in each cluster and their count
keywords = {}
counts = {}
for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent = [word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)  # frequency distribution of the words
    keywords[cluster] = nlargest(50, freq, key=freq.get)  # top 100 largest elements
    counts[cluster] = freq

In [201]:
# top keywords unique to each cluster
unique_keys = {}
for cluster in range(3):
    other_cluster = list(set(range(3)) - set([cluster]))
    
    keys_other_cluster = set(keywords[other_cluster[0]]).union(set(keywords[other_cluster[1]]))
    unique = set(keywords[cluster]) - keys_other_cluster  # SET(A) - SET(B)
    unique_keys[cluster] = nlargest(30, unique, key=counts[cluster].get)
print unique_keys
# 0 is for latest techs, 1 is for funding and investment, 2 stocks

{0: ['apple', 'qualcomm', 'drone', 'china', 'industry', 'times', 'drones', 'products', 'consumer', 'app', 'kalanick', 'drivers', 'february', 'high', 'still', 'electronics', 'australia', '300', 'chips', '2017', 'months', 'maker', 'even', 'group', 'iphones', 'iphone', 'dyson', 'devices'], 1: ['revenue', 'quarter', 'shares', 'facebook', 'investors', 'share', 'snap', 'stock', 'growth', 'snapchat', 'reality', 'ipo', 'customers', 'users', 'ad', 'trading', 'rose', 'analysts', 'per', 'next', 'earnings', 'reported', 'price', 'fell'], 2: ['india', 'flipkart', 'softbank', 'snapdeal', 'tesla', 'much', 'money', 'son', 'going', 'ebay', 'tech', 'cars', 'e-commerce', 'investment', 'amazon', 'self-driving', 'long', 'largest', 'get', 'valuation', 'years', 'start', 'take', 'may', 'fund', 'country', 'software']}


# Classification

In [202]:
article = "Infosys Ltd, despite a relatively good performance over the past two years, is battling perceptions of under-performance because of its inability to keep pace with its ambitious target of more than doubling its revenue to $20 billion by March 2021. This target appears to have done more harm than good: analysts continue to rate the company using it as a metric; and people in the know attribute the departure of at least a few senior executives over the past few years to it. Now, an analyst at a foreign brokerage has suggested that the company is better off without the target. “Management continues to maintain ‘aspirational’ revenue and margin targets that are increasingly becoming unlikely to be achieved, in our opinion. Given Infy (Infosys) is now guiding to lower levels of both revenue growth and margins in FY18 (2017-18), we think management would be well-served to either amend or drop its aspirational targets,” Keith Bachman, an analyst with BMO Capital Markets, wrote in a note dated 16 April. Since outlining this target, Sikka steered Infosys to dollar revenue growth of 9.1% in 2015-16 and 7.4% in 2016-17; for the second consecutive year, Infosys will grow faster than its larger rival Tata Consultancy Services Ltd (TCS) and cross-town peer Wipro Ltd (Wipro). Still, Infosys continues to be evaluated on the progress it makes in becoming a $20 billion company. Worryingly for the management, a bigger fallout of the targets has been the pressure on senior leaders, making a few of them leave. Since April 2015, half-a-dozen executive vice-presidents (EVPs) and a dozen senior vice-presidents (SVPs) have quit. Not all departures are on account of the targets but two former executives admit that they left Infosys as they could not keep up with the “impossible targets.”"

In [203]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(X, km.labels_)  # training phase

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [204]:
test = vectorizer.transform([article.decode('utf8').encode('ascii', errors='ignore')]) # represent in tf-idf

In [205]:
test

<1x1706 sparse matrix of type '<type 'numpy.float64'>'
	with 76 stored elements in Compressed Sparse Row format>

In [207]:
classifier.predict(test)  # testing phase

array([1], dtype=int32)

In [216]:
# https://www.cnbc.com/2017/12/04/jpmorgan-investors-should-rotate-into-these-sectors-with-gop-tax-plan.html
article2 = """With the Republican tax plan headed to the bicameral conference committee, JPMorgan is encouraging investors to make some tweaks to their portfolios.

A reduced corporate tax rate like that proposed in the GOP tax plan may be just the type of catalyst the market needs to spark rotation into value and away from growth stocks, according to strategist Dubravko Lakos-Bujas.

"We viewed the U.S. Tax legislation as the most significant upside catalyst for equities given the market was underpricing the probability of passage," explained Lakos-Bujas in Monday's note to clients. "While the tax trades have been gradually moving higher since the House passed its bill mid-November, the rotation across styles, sectors, and specialized portfolios accelerated on legislative progress last week. In fact, some of the moves appeared panicked."
"""

In [217]:
test2 = vectorizer.transform([article2.decode('utf8').encode('ascii', errors='ignore')]) # represent in tf-idf

In [218]:
test2

<1x1706 sparse matrix of type '<type 'numpy.float64'>'
	with 32 stored elements in Compressed Sparse Row format>

In [219]:
classifier.predict(test2)

array([2], dtype=int32)